In [1]:
import os
import sys
import pickle

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xarray as xr
import xesmf as xe
import xcdat as xc
import xsearch as xs
import xskillscore as xscore

from glob import glob 
from global_land_mask import globe
from typing import List, Tuple, Dict, Union, Optional, Any, Callable, Iterable, Sequence, cast
from scipy.stats import linregress

# Ignore xarray warnings (bad practice)
import warnings
warnings.simplefilter("ignore") 

In [2]:
os.chdir('/home/espinosa10/tropical_pacific_clouds')

In [3]:
def ingest_and_process(
    output_grid: np.ndarray,
    var: str = "tos", 
    cmipTable: str ="Omon", 
    era: str = "CMIP6",
    testing: bool = False,
    calc_anoms: bool = False
) -> dict:
    """
    Ingests all the data from the CMIP piControl experiment. 
    Optionally remove the seasonal cycle and detrend the data, regrid to a commond 2.5 x 2.5 degree grid and return a new dataset

    Returns:
        xarray datasets with dimensions (model, time, lat, lon)
    """

    # Specify the ensemble member
    if era == "CMIP6": 
        member = "r1i1p1f1"
    else:
        member = "r1i1p1"

    # Find all the paths to the data
    dpaths = xs.findPaths(
        experiment="abrupt4xCO2",
        variable=var,
        frequency="mon",
        cmipTable=cmipTable,
        mip_era=era, 
        activity="CMIP",
        member=member
    )
    models =  xs.getGroupValues(dpaths, 'model')
    print("Models: ", len(models), models)
    dpaths = list(dpaths.keys())
    
    # Create an empty Dataset
    ds = []
    valid_models = []

    for i, (model_path, model) in enumerate(zip(dpaths, models)):
        print("Starting model: ", model)
        print("Model path: ", model_path)

        try:
            # Load data
            da = xc.open_mfdataset(glob(model_path + "/*.nc")) #, parallel=True, chunks="auto")
            if (var == "ta"):
                da = da.sel(plev=70000)
            if (var in ["hur", "ua", "va", "zg"]): 
                # da = da.sel(plev=100000)
                da = da.sel(plev=85000)

            # Regrid Data
            output = da.regridder.horizontal(var, output_grid, tool='xesmf', method='bilinear')

            # Deseasonalize (no need to detrend piControl data, there should be no drift)
            if calc_anoms:
                output = output.temporal.departures(var, "month")

            ntime, _, _ = output[var].shape

            # nmonths = 165*12 # historical
            # nmonths = (2100-2015)*12 # piControl
            nmonths = 150*12
            if ntime < nmonths:
                continue
            
            output = output[var][:nmonths]
            time = np.arange(np.datetime64("2015-01"), np.datetime64("2015-01") + np.timedelta64(nmonths, 'M'), dtype="datetime64[M]")
            output["time"] = time
            coords = output.coords.keys()
            print(coords)
            if "height" in coords:
                output = output.drop('height')
            ds.append(output)
            valid_models.append(model)

            # Only load one model if testing
            if testing:
                _, axes = plt.subplots(ncols=2, figsize=(16, 4))
                da[var].isel(time=0).plot(ax=axes[0])
                axes[0].set_title('Input data')
                output.isel(time=0).plot(ax=axes[1])
                axes[1].set_title('Output data')
                plt.tight_layout()

                if i == 1:
                    break
        
        except Exception as e:
            print("Model failed: ", model, e)

    ds = xr.concat(ds, dim='model', coords='minimal', compat='override')
    ds.coords['model'] = list(valid_models)

    return ds


def collect_data():
    """
    Iterate through all the variables and eras and save the data
    """

    # Define Constants
    eras = ["CMIP5"]
    # variables = ["rsutcs", "rsut", "tos"] # Shortwave cloud forcing
    # variables = ["hur", "tas", "psl", "ta"] # EIS 
    # variables = ["rlut", "rlutcs"] # Longwave cloud forcing
    variables = [
        # "hur"
        # "rlds", # surface downwelling longwave flux, all sky
        # "rlus", # surface upwelling longwave flux, all sky
        # "rldscs", # surface downwelling longwave flux, clear sky
        # "rsds", # surface downwelling shortwave flux, all sky
        # "rsdscs", # surface downwelling shortwave flux, clear sky
        # "rsus", # surface upwelling shortwave flux, all sky (probably don't need)
        # "rsuscs", # surface upwelling shortwave flux, clear sky (probably don't need)
        # "hfls", # surface upward latent heat flux
        # "hfss", # surface upward shortwave heat flux
        # "sfcWind", # surface wind speed
        # "uas", # zonal 10 meter wind speed 
        # "vas", # meridional 10 meter wind speed
        "psl", # Mean sea-level pressure
        # "ua", # zonal wind speed (lowest level bc not all models have 10m wind speed)
        # "va" # meridional wind speed (lowest level bc not all models have 10m wind speed)
        # "zg"
        # "tauu", # zonal wind stress
        # "tauv", # meridional wind stress
        # "tas", # surface air temperature
        # "pr", # precipitation flux (km m^-2 s^-1)
        # "siconc",
        # "sic",
    ]
    calc_anoms = False

    # Create output grid
    lat = np.arange(-88.75, 90, 2.5)
    lon = np.arange(1.25, 360, 2.5)
    # lat = np.arange(-89.5, 90.5, 1)
    # lon = np.arange(.5, 360.5, 1)
    output_grid = xc.create_grid(lat, lon)

    for era in eras: 
        print("Starting era: ", era)
        for var in variables:
            if var == "sfcWind" and era == "CMIP6": continue

            print("Starting variable: ", var)

            if var == "tos": 
                table = "Omon"
            elif var in ["siconc", "sic"]:
                if var == "siconc": 
                    era = "CMIP6"
                    table = "SImon"
                if var == "sic": 
                    era  = "CMIP5"
                    table = "OImon"
            else: 
                table = "Amon"

            ds = ingest_and_process(
                var=var,
                cmipTable=table,
                era=era,
                testing=TESTING,
                output_grid=output_grid,
                calc_anoms=False
            )

            print(ds)
            if (var == "hur") or (var == "ta"):
                var = f"{var}-surface"
                
            if calc_anoms:
                ds.to_netcdf(f"{var}_mon_{era}_4xC02.nc")
                ds.close()
            else: 
                ds.to_netcdf(f"{var}_mon_{era}_4xC02.nc")
                ds.close()


TESTING = False
collect_data()

Starting era:  CMIP5
Starting variable:  psl
Models:  31 ['CNRM-CM5-2', 'CNRM-CM5', 'bcc-csm1-1', 'bcc-csm1-1-m', 'GFDL-ESM2G', 'GFDL-ESM2M', 'GFDL-CM3', 'FGOALS-g2', 'EC-EARTH', 'FGOALS-s2', 'MRI-CGCM3', 'IPSL-CM5B-LR', 'IPSL-CM5A-MR', 'IPSL-CM5A-LR', 'CCSM4', 'HadGEM2-ES', 'ACCESS1-3', 'GISS-E2-H', 'GISS-E2-R', 'CSIRO-Mk3-6-0', 'CanESM2', 'inmcm4', 'BNU-ESM', 'MIROC-ESM', 'MIROC5', 'MPI-ESM-MR', 'MPI-ESM-P', 'MPI-ESM-LR', 'ACCESS1-0', 'NorESM1-M', 'GFDL-ESM2G-1']
Starting model:  CNRM-CM5-2
Model path:  /p/css03/cmip5_css01/data/cmip5/output1/CNRM-CERFACS/CNRM-CM5-2/abrupt4xCO2/mon/atmos/Amon/r1i1p1/v20130402/psl/
Starting model:  CNRM-CM5
Model path:  /p/css03/cmip5_css01/data/cmip5/output1/CNRM-CERFACS/CNRM-CM5/abrupt4xCO2/mon/atmos/Amon/r1i1p1/v20110701/psl/
KeysView(Coordinates:
  * time     (time) datetime64[ns] 2015-01-01 2015-02-01 ... 2164-12-01
  * lat      (lat) float64 -88.75 -86.25 -83.75 -81.25 ... 83.75 86.25 88.75
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ..